In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import unidecode
import spacy

c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
top = pd.read_csv('top200.csv')

bottom = pd.read_csv('bottom200.csv')

In [3]:
def chorus(df):
    df['chorus'] = ''
    i = 0
    for morceau in df['lyrics']:
        split = str(morceau).split("Refrain")
        if len(split) == 1:
            split = str(morceau).split("Chorus")
        if len(split) == 1:
            df['chorus'][i] = ''
        else:
            chorus_and_co = split[1].split('] ')
            if len(chorus_and_co) == 1:
                df['chorus'][i] = ''
            else:
                chorus = chorus_and_co[1].split(' [')
                df['chorus'][i] = chorus[0]
        i+=1
    return df

In [4]:
top = chorus(top)
bottom = chorus(bottom)

C:\Users\bapti\AppData\Local\Temp\ipykernel_19068\1816682452.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = chorus[0]
C:\Users\bapti\AppData\Local\Temp\ipykernel_19068\1816682452.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = ''
C:\Users\bapti\AppData\Local\Temp\ipykernel_19068\1816682452.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chorus'][i] = chorus[0]
C:\Users\bapti\AppDat

In [5]:
top['lyrics'].head()

0    [Paroles de "TOUT VA BIEN" ft. Naps & Ninho] [...
1    [Paroles de "FADE UP" ft. Hamza & SCH] [Intro ...
2    [Paroles de "DIE"] [Intro] La mala est gangx E...
3    [Paroles de "PETETE"] [Intro] Pew Grr Pew pew ...
4    [Paroles de "Time Time"] [Couplet 1 : Squeezie...
Name: lyrics, dtype: object

In [6]:
bottom['lyrics'].head()

0    [Paroles de "BUMPY"] [Couplet 1] Mourir du P38...
1    [Paroles de "VALLON DES AUFFES"] [Intro] HuFel...
2    [Paroles de "BISON"] [Intro] Breh breh [Couple...
3    [Paroles de "IMPACT"] [Intro] Gang gang gang g...
4    [Paroles de "BOSS"] [Intro] Nardey c'est chaud...
Name: lyrics, dtype: object

In [7]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

In [8]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [15]:
def add_lyrics_features(df):
    df['lyrics']=df['lyrics'].dropna()
    df['lyrics_nb_max'] = 0
    df['lyrics_nb_moy'] = 0
    i = 0
    while i < len(df['lyrics']):
        
        df['lyrics'][i] = str(df['lyrics'][i])

        if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
            df['lyrics'][i] = unidecode.unidecode(df['lyrics'][i])
            df['lyrics'][i] = re.sub(r'\[.*?\]', '', df['lyrics'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['lyrics'][i] = df['lyrics'][i].replace(c, ' ')
            df['lyrics'][i] = df['lyrics'][i].lower()
            df['lyrics'][i] = df['lyrics'][i].split(' ')
            df['lyrics'][i] = stop_words_english(df['lyrics'][i])
            df['lyrics'][i] = stop_words_french(df['lyrics'][i])

            if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
                d = {j:df['lyrics'][i].count(j) for j in df['lyrics'][i]}

                max = np.array(list(d.values())).max()
                mean = np.array(list(d.values())).mean()

                df['lyrics_nb_max'][i] = max
                df['lyrics_nb_moy'][i] = mean

        i +=1
    return df

In [10]:
def add_chorus_features(df):
    df['chorus']=df['chorus'].dropna()
    df['nb_words_chorus'] = 0
    df['nb_different_words_chorus'] = 0
    df['diff_duplicates_chorus'] = 0
    i = 0
    while i < len(df['chorus']):
        if (df['chorus'][i] != 'nan') and (len(df['chorus'][i]) > 0):
            df['chorus'][i] = unidecode.unidecode(df['chorus'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['chorus'][i] = df['chorus'][i].replace(c, ' ')
            df['chorus'][i] = df['chorus'][i].lower()
            df['chorus'][i] = df['chorus'][i].split(' ')
            df['chorus'][i] = stop_words_english(df['chorus'][i])
            df['chorus'][i] = stop_words_french(df['chorus'][i])

            nb_words = len(df['chorus'][i])
            without_duplicates = len(list(set(df['chorus'][i])))
            diff = nb_words - without_duplicates

            df['nb_words_chorus'][i] = nb_words
            df['nb_different_words_chorus'][i] = without_duplicates
            df['diff_duplicates_chorus'][i] = diff

        i +=1
    return df

In [11]:
top = add_lyrics_features(top)
top = add_chorus_features(top)
top.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,summer,autumn,id,chorus,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
0,TOUT VA BIEN (feat. Ninho & Naps),Alonzo,rap,1,0.660,0.720,3.0,-5.874,0.0,0.0753,...,1,1,6OZwia8loN0aPS0vTvsBjR,"[cherie, veut, yves, saint, lau, lau, donne, c...",53,2.489362,4.692860,45,30,15
1,FADE UP,ZEG P,autre,2,0.758,0.599,4.0,-5.990,0.0,0.0877,...,1,1,4ZpIuzx91EAPK3VimONbfB,"[fade, fade, nan, fade, fade, nan, fade, fade,...",32,2.637500,4.065845,12,2,10
2,DIE,Gazo,rap,3,0.695,0.630,8.0,-7.160,0.0,0.0350,...,1,1,3D29kjUyWxsT3jUUTtARVQ,"[crois, evite, alors, maille, maille, veut, sa...",14,2.200000,2.241428,34,19,15
3,PETETE,Gambi,rap,4,0.752,0.669,10.0,-9.817,0.0,0.2510,...,1,1,0z3bi63SNZ5ylyHOzb81Uq,"[cr, partout, bloc, okay, khabat, vite, pete, ...",14,2.208633,2.474255,38,27,11
4,Time Time,Trei Degete,autre,5,0.715,0.799,0.0,-6.447,0.0,0.0433,...,0,1,5wKDPtbdggE1roeVp3UdXX,"[dansam, time, time, dansam, time, time, asa, ...",160,13.423077,30.460786,67,12,55


In [12]:
bottom = add_lyrics_features(bottom)
bottom = add_chorus_features(bottom)
bottom.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,summer,autumn,id,chorus,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
0,BUMPY,Alonzo,rap,1,0.713,0.820,5.0,-6.650,0.0,0.3670,...,1,1,1D8lLfcmm2kpW7CTdhrFN9,"[avoir, vie, bumpy, coute, cher, cette, vida, ...",11,2.021277,1.798219,50,24,26
1,VALLON DES AUFFES,Alonzo,rap,2,0.644,0.798,5.0,-5.791,0.0,0.0961,...,1,1,0mJHGKoDHfSkDxvzPbo6EV,"[pose, bas, chez, vis, nuit, fuis, jour, pah, ...",60,2.669118,6.849276,43,22,21
2,BISON,Alonzo,rap,3,0.838,0.693,4.0,-6.547,0.0,0.2610,...,1,1,2JkWkGL2OQHUpAdyRaH5a8,"[vrais, putas, tout, melange, melange, melange...",24,2.480315,3.493189,56,15,41
3,IMPACT,Gazo,rap,4,0.767,0.673,8.0,-5.843,0.0,0.2280,...,1,1,1hlUHJgbdFdvZvG64a3W1g,"[veux, pe, mala, gang, gang, si, perds, impact...",14,2.830000,1.755306,34,30,4
4,BOSS,Gazo,rap,5,0.708,0.676,4.0,-8.016,0.0,0.1390,...,1,1,2K2hSiCZAwwlFvwTxVRqoi,"[revend, coke, bosse, vrais, machines, jamais,...",13,2.781818,2.595037,59,35,24


In [13]:
top.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,winter,spring,summer,autumn,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
count,1320.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,...,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000
mean,660.500000,0.721116,0.646079,5.282790,-7.003566,0.391964,0.175331,0.291488,0.007608,0.153380,...,0.373485,0.359848,0.319697,0.491667,21.337121,2.221953,2.843931,28.550000,16.093182,12.456818
std,381.195488,0.128060,0.147248,3.572734,2.246327,0.488374,0.137636,0.223102,0.058262,0.104487,...,0.483913,0.480138,0.466536,0.500120,18.439315,1.121756,2.359563,21.964178,13.279110,14.308513
min,1.000000,0.234000,0.120000,0.000000,-21.189000,0.000000,0.025800,0.000307,0.000000,0.023200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,330.750000,0.641500,0.549000,2.000000,-8.147500,0.000000,0.056800,0.110500,0.000000,0.092800,...,0.000000,0.000000,0.000000,0.000000,10.000000,1.683728,1.517463,12.750000,6.000000,1.000000
50%,660.500000,0.742000,0.659000,6.000000,-6.670000,0.000000,0.129000,0.247000,0.000000,0.115000,...,0.000000,0.000000,0.000000,0.000000,16.000000,2.026709,2.259899,26.500000,14.000000,8.000000
75%,990.250000,0.813500,0.754000,8.000000,-5.531500,1.000000,0.271500,0.420000,0.000043,0.173000,...,1.000000,1.000000,1.000000,1.000000,26.000000,2.561181,3.461119,43.000000,24.000000,18.000000
max,1320.000000,0.973000,0.965000,11.000000,-2.018000,1.000000,0.828000,0.994000,0.956000,0.812000,...,1.000000,1.000000,1.000000,1.000000,160.000000,13.423077,30.460786,165.000000,119.000000,139.000000


In [14]:
bottom.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,winter,spring,summer,autumn,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std,nb_words_chorus,nb_different_words_chorus,diff_duplicates_chorus
count,1053.00000,1051.000000,1051.000000,1051.00000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,...,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000,1053.000000
mean,527.00000,0.667426,0.592365,5.16746,-7.984419,0.472883,0.165137,0.335532,0.034373,0.162281,...,0.484330,0.450142,0.421652,0.592593,17.718898,1.981400,2.491815,21.449193,11.733143,9.716049
std,304.11922,0.166181,0.179246,3.57485,3.101373,0.499502,0.152667,0.268749,0.152795,0.117524,...,0.499992,0.497744,0.494058,0.491585,15.896253,0.986738,2.023122,19.619250,11.281399,12.227101
min,1.00000,0.063300,0.034600,0.00000,-25.521000,0.000000,0.024100,0.000030,0.000000,0.026100,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,264.00000,0.567000,0.501000,2.00000,-9.037500,0.000000,0.047250,0.111000,0.000000,0.096600,...,0.000000,0.000000,0.000000,0.000000,8.000000,1.568182,1.255775,0.000000,0.000000,0.000000
50%,527.00000,0.705000,0.615000,5.00000,-7.265000,0.000000,0.101000,0.266000,0.000000,0.116000,...,0.000000,0.000000,0.000000,1.000000,14.000000,1.929412,2.092865,18.000000,10.000000,4.000000
75%,790.00000,0.788000,0.713000,8.00000,-6.088500,1.000000,0.256000,0.511000,0.000107,0.186000,...,1.000000,1.000000,1.000000,1.000000,23.000000,2.428571,3.270236,34.000000,18.000000,17.000000
max,1053.00000,0.973000,0.968000,11.00000,-2.021000,1.000000,0.956000,0.992000,0.965000,0.848000,...,1.000000,1.000000,1.000000,1.000000,123.000000,7.756098,16.216551,100.000000,61.000000,82.000000


In [ ]:
def lemmatization(nlp, texte):
    i = 0
    for mot in texte:
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
            
    return texte

In [ ]:
nlp_fr = spacy.load('fr_core_news_md')
nlp_en = spacy.load('en_core_web_md')

In [ ]:
i = 0
while i < len(top['lyrics']):
    if len(top['lyrics'][i]) > 0:
        top['lyrics'][i] = lemmatization(nlp_fr, top['lyrics'][i])
        top['lyrics'][i] = lemmatization(nlp_en, top['lyrics'][i])
    i +=1